In [3]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

from pathlib import Path
import platform
import pandas as pd
import numpy as np
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
from matplotlib import dates, ticker

from ladybug.epw import EPW
import oshe as tc

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
if platform.system() == "Darwin":
    idd_file = "/Applications/OpenStudio-2.9.0/EnergyPlus/Energy+.idd"
    epw_file = "/Users/tristan/Desktop/github/oshe/example/weatherfile.epw"
else:
    idd_file = r"C:\openstudio-2.9.0\EnergyPlus\Energy+.idd"
    epw_file = r"C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\UWG_ARE_AZ_Abu.Dhabi-Bateen.Exec.AP.412160_TMYx_URBAN_02.epw"

In [8]:
# Load weatherfile data

# Load epw
epw = tc.helpers.load_weather(epw_file)

# Get climate variables
dbt = epw.dbt.values
rh = epw.rh.values
ws = epw.ws.values
hir = epw.hir.values

# Calculate hourly solar altitudes
sun_altitude = epw.sun_altitude.values

In [27]:
# NV method processing

_dir = Path("C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\CapitalGardens")

# Load Openfield MRT and calculate UTCI using same method as NV MRT > UTCI is generated
of_mrt = pd.read_csv(r"C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\openfield_nv.mrt", header=None).values.T
of_utci = tc.utci.universal_thermal_climate_index(dbt, of_mrt, ws, rh)[0]

# Load MRT
mean_radiant_temperature = pd.read_csv(_dir+"\MRT.csv", header=None).values.T

# Load PTS
points = pd.read_csv(_dir+"\points.csv", header=None).values

# Load focus pts
focus_pts = pd.read_csv(_dir + "\\focus_pts.csv", header=None).values.T[0]

# Set radiance geometries
rad_files = []
for root, dirs, files in os.walk(_dir):
    for file in files:
        if file.endswith("opq.rad") or file.endswith("glz.rad"):
             rad_files.append(os.path.join(root, file))

# Calculate UTCI
universal_thermal_climate_index = tc.utci.utci_parallel(
        threads=8, 
        air_temperature=dbt, 
        mean_radiant_temperature=mean_radiant_temperature.T, 
        wind_speed=ws, 
        relative_humidity=rh
    )

# Create UTCI object
utci = tc.plot.UTCI(of_utci, universal_thermal_climate_index, points)

# Plot the results
utci.generate_plots(rad_files=rad_files, focus_pts=focus_pts, boundary=None, plot_directory=_dir+"\plots2")

Universal thermal climate index calculated [0.10 seconds]
Plot saved to C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\ChamberOfCommerce\plots2\Area2\openfield_comfortheatmap.png
Plot saved to C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\ChamberOfCommerce\plots2\Area2\comfortable_hours_may.png
Plot saved to C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\ChamberOfCommerce\plots2\Area2\comfortable_hours_october.png
Plot saved to C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\ChamberOfCommerce\plots2\Area2\comfortable_hours_annual.png
Plot saved to C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\ChamberOfCommerce\plots2\Area2\context.png
Plot saved to C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\ChamberOfCommerce\plots2\Area2\context_ptlabel.png
Plot saved to C:\Users\tgerrish\OneDrive - BuroHappold\0045131 Ghadan Future Parks\ChamberOfCommerce\plots2\Area

In [31]:
import tempfile

eso_file = Path(tempfile.gettempdir()) / "openfield/ground_surface_temperature/eplusout.eso"
gnd = tc.geometry.Ground(material=tc.material.material_dict["CONCRETE"], xy=6, depth=1.5, subsurface_size=3)

print(eso_file)
tc.energyplus.run_energyplus(epw_file, idd_file, ground=gnd, run=True)


/var/folders/gk/bfbckh990jl1mgl1m5864_fc0000gn/T/openfield/ground_surface_temperature/eplusout.eso
Ground surface temperature simulation completed


FileNotFoundError: [Errno 2] File /var/folders/gk/bfbckh990jl1mgl1m5864_fc0000gn/T/openfield/ground_surface_temperature/eplusout.csv does not exist: '/var/folders/gk/bfbckh990jl1mgl1m5864_fc0000gn/T/openfield/ground_surface_temperature/eplusout.csv'

In [40]:
str(eso_file)

'/var/folders/gk/bfbckh990jl1mgl1m5864_fc0000gn/T/openfield/ground_surface_temperature/eplusout.eso'

In [99]:
class EsoFile(object):
    def __init__(self, eso_file: str):
        self.eso_file = eso_file
        self.objects = None
        self.variables = None
        self.values = None
        self.reporting_frequency = None
        self.version = None
        self.creation_date = None
        
        self.read_eso()
    
    def read_eso(self):
        
        self.objects = []
        self.variables = []
        self.reporting_frequency = []
        self.values = []
        
        with open(self.eso_file, "r") as f:
            data = [i.strip() for i in f.readlines()]
            
            # Get metadata
            self.version = data[0].split(", ")[1].replace("Version ", "")
            self.creation_date = pd.to_datetime(data[0].split(", ")[2].replace("YMD=", ""))
            
            # Get reporting frequency and variables
            for i in data:
                if "!" in i:
                    if "! " in i:
                        pass
                    else:
                        # Reporting frequency
                        self.reporting_frequency.append(i.split(" !")[-1])
                        # Object
                        self.objects.append(i.split(" !")[0].split(",")[2])
                        # Variable
                        self.variables.append(i.split(" !")[0].split(",")[3])
            
            # Get the values
            n_vals = len(self.reporting_frequency)
            split_idx = [n for n, i in enumerate(data) if i == "End of Data Dictionary"][0] + 2
            data = [i.split(",")[1] for i in data[split_idx:-2] if len(i.split(",")) <= 2]
            self.values = np.array(tc.helpers.chunks(data, n=n_vals)).astype(np.float64)

    
EsoFile(eso_file).values


array([[21.90594974, 21.90594974, 21.90594974, ..., 24.6143621 ,
        24.6143621 , 24.6143621 ],
       [24.38764736, 24.38764736, 24.38764736, ..., 54.42630747,
        54.42630747, 54.42630747],
       [57.88793838, 57.88793838, 57.88793838, ..., 44.38270252,
        44.38270252, 44.38270252],
       [41.23612757, 41.23612757, 41.23612757, ..., 26.57495083,
        26.57495083, 26.57495083]])

In [11]:
# SC method processing

# Open-field MRT and UTCI calculation
of = tc.openfield.openfield(epw_file=epw_file, idd_file=idd_file, material="CONCRETE", shaded=True)
of_mrt = of.mrt.values
of_utci = of.utci.values

Ground surface temperature simulation completed


FileNotFoundError: [Errno 2] File /var/folders/gk/bfbckh990jl1mgl1m5864_fc0000gn/T/openfield/ground_surface_temperature/eplusout.csv does not exist: '/var/folders/gk/bfbckh990jl1mgl1m5864_fc0000gn/T/openfield/ground_surface_temperature/eplusout.csv'

In [14]:
# Load case recipe
_dir = Path(r"C:\Users\tgerrish\Desktop\_0Ghadan21\STREETFACELIFTS_P1")
reload = False

# Create output directory for generated images
plot_directory = tc.helpers.create_directory(_dir / "plots2")

# Load radiance geometries
rad_files = tc.helpers.flatten([tc.helpers.find_files(_dir, endswith=i) for i in ["glz.rad", "opq.rad"]])

# Load points
points = pd.read_csv(_dir / "_sample_points.xyz", header=None).values

# Load plot tricontour plot 
boundary = pd.read_csv(_dir / "_boundary_points.xyz", header=None).values[:, :2]

# Load focus pts
focus_pts = pd.read_csv(_dir / "focus_pts.csv", header=None).values.T[0]

# Load ground reflectivities
ground_reflectivities = pd.read_csv(_dir / "_ground_reflectivities.rfl", header=None).values

# Load surface view factors
surface_view_factors = pd.read_csv(_dir / "_surface_view_factors.vf", header=None).values

# Load sky view factors
sky_view_factors = pd.read_csv(_dir / "_sky_view_factors.vf", header=None).values / 100  # values output from GH are in %, need to be in decimal

if not reload:
    # Read EnergyPlus annual hourly surface temperature results
    surface_temperatures = tc.helpers.load_energyplus_results(_dir / "OpenStudio" / _dir.name / "ModelToIdf/eplusout.csv")

    # Read Radiance simulation annual hourly incident radiation results
    radiation_direct, radiation_diffuse = tc.helpers.load_radiance_results(_dir / "gridbased_annual/result")
    
    # Calculate generic sky temperature
    skytemp = tc.oshe.sky_temperature(hir)

    # Append sky temperature and air temperature onto end of surface temperatures
    all_srf_temps = np.vstack([surface_temperatures, skytemp, dbt])

    # Append sky view factor to end of surface view factors
    all_vfs = np.hstack([surface_view_factors, sky_view_factors])

    # Add view factor to end of each group, bringing total to 1
    makeups = np.where(1 - all_vfs.sum(axis=1) <= 0, 0, 1 - all_vfs.sum(axis=1))
    all_vfs = np.vstack([all_vfs.T, makeups]).T

    # Normalise view factors to 0-1
    all_vfs = all_vfs.T * (1 / all_vfs.sum(axis=1))

    # Calculate overall surrounding surface temperature using view factors and individual surface temperatures
    surrounding_surface_temperatures = np.power(np.matmul(all_vfs.T, np.power(all_srf_temps.T + 273.15, 4).T), 0.25) - 273.15
    print("View-factored surface temperatures calculated")


EnergyPlus results loaded
Radiance results loaded
View-factored surface temperatures calculated


In [15]:
# Calculate MRT and write to file

mrt_path = _dir / "result.mrt"
if not reload:
    mean_radiant_temperature = tc.mrt.mrt_parallel(
        threads=8, 
        surrounding_surfaces_temperature=surrounding_surface_temperatures, 
        horizontal_infrared_radiation_intensity=hir, 
        diffuse_horizontal_solar=radiation_diffuse, 
        direct_normal_solar=radiation_direct, 
        sun_altitude=sun_altitude, 
        ground_reflectivity=ground_reflectivities, 
        sky_exposure=sky_view_factors
    )

    # Write to file
    df_mrt = pd.DataFrame(mean_radiant_temperature.T)
    df_mrt.round(6).to_csv(mrt_path, index=False)
    print("MRT calculated and written to {}".format(mrt_path))
else:
    mean_radiant_temperature = pd.read_csv(mrt_path).values.T
    print("MRT reloaded from {}".format(mrt_path))

MRT calculated and written to C:\Users\tgerrish\Desktop\_0Ghadan21\STREETFACELIFTS_P1\result.mrt


In [16]:
# Calculate UTCI and write to file

utci_path = _dir / "result.utci"
if not reload:
    universal_thermal_climate_index = tc.utci.utci_parallel(
        threads=8, 
        air_temperature=dbt, 
        mean_radiant_temperature=mean_radiant_temperature.T, 
        wind_speed=ws, 
        relative_humidity=rh
    )

    # Write to file
    df_utci = pd.DataFrame(universal_thermal_climate_index.T)
    df_utci.round(6).to_csv(utci_path, index=False)
    print("UTCI calculated and written to {}".format(utci_path))
else:
    universal_thermal_climate_index = pd.read_csv(utci_path).values.T
    print("UTCI reloaded from {}".format(utci_path))

UTCI calculated and written to C:\Users\tgerrish\Desktop\_0Ghadan21\STREETFACELIFTS_P1\result.utci


In [19]:
# Construct results object
utci = tc.plot.UTCI(of_utci, universal_thermal_climate_index, points)

# Plot the results
utci.generate_plots(rad_files=rad_files, boundary=boundary, plot_directory=plot_directory, focus_pts=focus_pts)

Plot saved to C:\Users\tgerrish\Desktop\_0Ghadan21\STREETFACELIFTS_P1\plots2\openfield_comfortheatmap.png
Plot saved to C:\Users\tgerrish\Desktop\_0Ghadan21\STREETFACELIFTS_P1\plots2\comfortable_hours_may.png
Plot saved to C:\Users\tgerrish\Desktop\_0Ghadan21\STREETFACELIFTS_P1\plots2\comfortable_hours_october.png
Plot saved to C:\Users\tgerrish\Desktop\_0Ghadan21\STREETFACELIFTS_P1\plots2\comfortable_hours_annual.png
Plot saved to C:\Users\tgerrish\Desktop\_0Ghadan21\STREETFACELIFTS_P1\plots2\context.png
Plot saved to C:\Users\tgerrish\Desktop\_0Ghadan21\STREETFACELIFTS_P1\plots2\context_ptlabel.png
Plot saved to C:\Users\tgerrish\Desktop\_0Ghadan21\STREETFACELIFTS_P1\plots2\context_focuspts.png
Plot saved to C:\Users\tgerrish\Desktop\_0Ghadan21\STREETFACELIFTS_P1\plots2\improvement_annual_morningshoulder.png
Plot saved to C:\Users\tgerrish\Desktop\_0Ghadan21\STREETFACELIFTS_P1\plots2\improvement_annual_afternoonshoulder.png
Plot saved to C:\Users\tgerrish\Desktop\_0Ghadan21\STREETFAC

C:\Users\tgerrish\Documents\GitHub\oshe\oshe\plot.py:290: UserWarning: No contour levels were found within the data range.
  levels=[-4, -3, -2, -1, 0, 1, 2, 3, 4], zorder=10, alpha=0.5)


Plot saved to C:\Users\tgerrish\Desktop\_0Ghadan21\STREETFACELIFTS_P1\plots2\reduction_may_morningshoulder.png
Plot saved to C:\Users\tgerrish\Desktop\_0Ghadan21\STREETFACELIFTS_P1\plots2\reduction_may_afternoonshoulder.png
Plot saved to C:\Users\tgerrish\Desktop\_0Ghadan21\STREETFACELIFTS_P1\plots2\reduction_may_morningafternoonshoulder.png
Plot saved to C:\Users\tgerrish\Desktop\_0Ghadan21\STREETFACELIFTS_P1\plots2\reduction_october_morningshoulder.png
Plot saved to C:\Users\tgerrish\Desktop\_0Ghadan21\STREETFACELIFTS_P1\plots2\reduction_october_afternoonshoulder.png
Plot saved to C:\Users\tgerrish\Desktop\_0Ghadan21\STREETFACELIFTS_P1\plots2\reduction_october_morningafternoonshoulder.png
Plot saved to C:\Users\tgerrish\Desktop\_0Ghadan21\STREETFACELIFTS_P1\plots2\pt0072_reductionheatmap.png
Plot saved to C:\Users\tgerrish\Desktop\_0Ghadan21\STREETFACELIFTS_P1\plots2\pt0072_comfortheatmap.png
Plot saved to C:\Users\tgerrish\Desktop\_0Ghadan21\STREETFACELIFTS_P1\plots2\pt0072_profile_